In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import numpy as np

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter

from qcodes.sweep import sweep, nest, chain, measurement, setter

C:\Users\a-sochat\AppData\Local\Continuum\Anaconda3\envs\qcodes\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
class Printer:
    def __init__(self, sweep_object): 
        self._table_list = sweep_object.parameter_table.table_list
        
        self._ind = set([tpl for d in self._table_list for tpl in d["independent_parameters"]])
        self._dep = set([tpl for d in self._table_list for tpl in d["dependent_parameters"]])
        
        ind_names = list(zip(*self._ind))[0] if len(self._ind) else []
        dep_names = list(zip(*self._dep))[0] if len(self._dep) else []
        
        self._all_param_names = np.append(ind_names, dep_names)
        
    def __enter__(self): 
        header_ind = "\t".join(["{} [{}]".format(*i) for i in self._ind])
        header_dep = "\t".join(["{} [{}]".format(*i) for i in self._dep])
        sep = " | "
        print((header_ind + sep + header_dep).strip(sep))
        
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        print(" " + "\t ".join(list([str(result.get(ip, "None")) for ip in self._all_param_names])))

In [5]:
x = ManualParameter("x", unit="V")
y = ManualParameter("y", unit="V")

m = ManualParameter("m", unit="A")
m.get = lambda: x() ** 2

n = ManualParameter("n", unit="A")
n.get = lambda: x() - y() ** 2 + 16

sweep_object = sweep(x, [0, 1, 2])(
    m,
    sweep(y, [0, 1, 2])(
        n  
    )
)

In [6]:
with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)

x [V]	y [V] | m [A]	n [A]
 0	 None	 0	 None
 0	 0	 None	 16
 0	 1	 None	 15
 0	 2	 None	 12
 1	 None	 1	 None
 1	 0	 None	 17
 1	 1	 None	 16
 1	 2	 None	 13
 2	 None	 4	 None
 2	 0	 None	 18
 2	 1	 None	 17
 2	 2	 None	 14


In [7]:
@measurement([("m3", "H"), ("m4", "H")])
def measurement_function(): 
    meas3 = int(np.random.normal(0, 1) * 10) / 10
    meas4 = int(np.random.normal(-5, 1) * 10) / 10
    return meas3, meas4

In [8]:
sweep_object = sweep(x, [0, 1, 2])(
    m,
    sweep(y, [0, 1, 2])(
        measurement_function  
    )
)

In [9]:
with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)

x [V]	y [V] | m [A]	m4 [H]	m3 [H]
 0	 None	 0	 None	 None
 0	 0	 None	 -4.2	 0.4
 0	 1	 None	 -4.7	 -0.5
 0	 2	 None	 -4.4	 -1.8
 1	 None	 1	 None	 None
 1	 0	 None	 -4.5	 -0.1
 1	 1	 None	 -3.9	 1.3
 1	 2	 None	 -3.8	 -0.4
 2	 None	 4	 None	 None
 2	 0	 None	 -4.0	 0.5
 2	 1	 None	 -6.8	 -0.4
 2	 2	 None	 -2.6	 0.4
